In [ ]:
SELECT *
FROM fengyingyang.braintree_disputes
LIMIT 50

In [ ]:
SELECT *
FROM risk_data_platform_prod.public.maindb_dispute_history
WHERE pgp_code = 'braintree'
LIMIT 50

In [ ]:
SELECT *
FROM payment_service_prod.public.maindb_pgp_disputes

LIMIT 50;

In [ ]:
SELECT *
FROM EDW.OPEX.FACT_FRAUD_DELIVERIES
WHERE pgp_code = 'braintree'
LIMIT 50;

# payment_service_prod.public.maindb_disputes & payment_service_prod.public.maindb_pgp_disputes

- payment_service_prod.public.maindb_disputes.id is exactly equal to payment_service_prod.public.maindb_pgp_disputes.dispute_id

- We should join on id = dispute_id to combine base + labeled data.

# fengyingyang.braintree_disputes & payment_service_prod.public.maindb_pgp_disputes

- COUNT of fengyingyang.braintree_disputes.dispute_id: 4261072
- Missing in payment_service_prod.public.maindb_pgp_disputes.pgp_resource_id:  129846
- 97% of fengyingyang.braintree_disputes.dispute_id are in payment_service_prod.public.maindb_pgp_disputes.pgp_resource_id
- mismatch in status/pgp_code (3805038(90%) of them match both)
- reason written in different ways: sequence of writing, fraud/fraudulance

QUESION:
1. do i need to deal with reason discrepancy this week

In [ ]:
SELECT *
FROM fengyingyang.braintree_disputes a
LEFT JOIN payment_service_prod.public.maindb_pgp_disputes b
  ON a.dispute_id = b.pgp_resource_id
WHERE b.pgp_resource_id IS NULL;

Example of mismatch in status and pgp_code

??  payment_service_prod.public.maindb_pgp_disputes has pgp_code:['braintree', 'jpm', 'paypal', 'stripe']
 In the file https://docs.google.com/document/d/1Ebkm-I025jMjmL3r-jBxf7hBDI_vakwJEa3Av4sY6ME/edit?tab=t.0 it says pgp are Braintree, Stripe, JPM, and PayPal belongs to Network)

- solve by just abandon them

In [ ]:
SELECT *
FROM payment_service_prod.public.maindb_pgp_disputes
WHERE pgp_resource_id = 'nyzt534ky8c5vw3r'
LIMIT 10;

In [ ]:
SELECT *
FROM fengyingyang.braintree_disputes
WHERE dispute_id = 'nyzt534ky8c5vw3r'
LIMIT 10;

Match count 90% on status and pgp_code, abandon the rest

Now, we come to the last part: reason

In [ ]:
SELECT COUNT(*) AS matched_status_count
FROM fengyingyang.braintree_disputes a
JOIN payment_service_prod.public.maindb_pgp_disputes b
  ON a.dispute_id = b.pgp_resource_id
WHERE a.status = b.status
  AND b.pgp_code = 'braintree'


discrepancy between reasons
95% fraud vs fraudulent
- what about general vs transaction_amount_differs, there are 9743 of them.

consider these below, 99.9% match

- AND NOT (a.reason = 'fraud' AND b.reason = 'fraudulent')
- AND NOT (a.reason = 'not_recognized' AND b.reason = 'unrecognized')
-  AND NOT (a.reason = 'product_unsatisfactory' AND b.reason = 'product_unacceptable')
- AND NOT (a.reason = 'cancelled_recurring_transaction' AND b.reason = 'subscription_canceled')
- AND NOT (a.reason = 'invalid_account' AND b.reason = 'incorrect_account_details')
-  AND NOT (a.reason = 'transaction_amount_differs' AND b.reason = 'general')

In [ ]:
SELECT a.dispute_id,
       a.reason AS bt_reason,
       b.reason AS pgp_reason,
       a.status,
       b.status
FROM fengyingyang.braintree_disputes a
JOIN payment_service_prod.public.maindb_pgp_disputes b
  ON a.dispute_id = b.pgp_resource_id
  WHERE a.status = b.status
  AND a.reason <> b.reason
  AND NOT (a.reason = 'fraud' AND b.reason = 'fraudulent')
  AND NOT (a.reason = 'not_recognized' AND b.reason = 'unrecognized')
  AND NOT (a.reason = 'product_unsatisfactory' AND b.reason = 'product_unacceptable')
  AND NOT (a.reason = 'cancelled_recurring_transaction' AND b.reason = 'subscription_canceled')
  AND NOT (a.reason = 'invalid_account' AND b.reason = 'incorrect_account_details')
  -- how should we consider these:
  AND NOT (a.reason = 'transaction_amount_differs' AND b.reason = 'general')

  --exclude equivalent mentioned above
  AND NOT (a.reason = 'fraud' or b.reason = 'fraudulent')
  AND NOT (a.reason = 'not_recognized' or b.reason = 'unrecognized')
  AND NOT (a.reason = 'product_unsatisfactory' or b.reason = 'product_unacceptable')
  AND NOT (a.reason = 'cancelled_recurring_transaction' or b.reason = 'subscription_canceled')
  AND NOT (a.reason = 'invalid_account' or b.reason = 'incorrect_account_details')



# risk_data_platform_prod.public.maindb_dispute_history & payment_service_prod.public.maindb_pgp_disputes

- no reason mismatch
- many status mismatch(why)

In [ ]:
SELECT h.dispute_id,
       h.status AS history_status,
       p.status AS pgp_status,
       h.reason AS history_reason,
       p.reason AS pgp_reason
FROM risk_data_platform_prod.public.maindb_dispute_history h
JOIN payment_service_prod.public.maindb_pgp_disputes p
  ON h.dispute_id = p.dispute_id
WHERE p.pgp_code = 'braintree'
AND h.pgp_code = 'braintree';


Compare status and reason between dispute_history and pgp_disputes.

Exclude:

Disputes created after Jan 1, 2024 where history_status ≠ "contested"  and   p.status='needs_response'

Disputes created before Jan 1, 2024 where status mismatch

In [ ]:
SELECT h.dispute_id,
       h.status AS history_status,
       p.status AS pgp_status,
       h.reason AS history_reason,
       p.reason AS pgp_reason,
       h.charge_date
FROM risk_data_platform_prod.public.maindb_dispute_history h
JOIN payment_service_prod.public.maindb_pgp_disputes p
  ON h.dispute_id = p.dispute_id
WHERE p.pgp_code = 'braintree'
  AND (
    -- Pre-2024: include only if status matches
    (h.charge_date < '2024-01-01' AND h.status = p.status)

    -- Post-2024: include only if contested and PGP status is meaningful
    OR (h.charge_date >= '2024-01-01' AND h.status = 'Contested' AND p.status <> 'needs_response')
  );


In [ ]:
-- no reason msmatch
SELECT COUNT(*) AS mismatch_count
FROM risk_data_platform_prod.public.maindb_dispute_history h
JOIN payment_service_prod.public.maindb_pgp_disputes p
  ON h.dispute_id = p.dispute_id
WHERE p.pgp_code = 'braintree'
  AND (
    h.reason <> p.reason
  );
